In [10]:
# notebook_training.ipynb

import pandas as pd
import numpy as np
import joblib  # Pour sauvegarder le modèle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error


In [11]:
# 1. Chargement
df = pd.read_csv("AER_credit_card_data.csv")

In [12]:
# 2. Encodage (Transformation des Yes/No en 1/0)
binary_cols = ['card', 'owner', 'selfemp']
for col in binary_cols:
    df[col] = df[col].map({'yes': 1, 'no': 0})

In [13]:
# 3. Séparation X et y
X = df.drop("expenditure", axis=1)
y = df["expenditure"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# 4. Création du Pipeline (Scaler + SVM)
# Le Pipeline est magique : il applique le Scaler automatiquement avant le SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVR())
])

In [15]:
# 5. Optimisation (GridSearch) - Demandé dans la consigne
param_grid = {
    'svm__C': [1, 10, 100, 1000],          # Paramètres de régularisation
    'svm__kernel': ['rbf', 'linear'],      # Types de noyaux
    'svm__epsilon': [0.1, 0.2, 0.5]        # Marge d'erreur
}

print("Démarrage de l'optimisation...")
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='r2', n_jobs=-1)
grid.fit(X_train, y_train)

print(f"Meilleurs paramètres : {grid.best_params_}")
print(f"Score R2 sur le test : {grid.score(X_test, y_test):.4f}")

Démarrage de l'optimisation...
Meilleurs paramètres : {'svm__C': 1000, 'svm__epsilon': 0.1, 'svm__kernel': 'rbf'}
Score R2 sur le test : 0.9150


In [16]:
# 6. Sauvegarde du modèle final
# On sauvegarde tout le pipeline (scaler inclus) dans un fichier .pkl
joblib.dump(grid.best_estimator_, 'model_svm.pkl')
print("Modèle sauvegardé sous 'model_svm.pkl' ✅")

Modèle sauvegardé sous 'model_svm.pkl' ✅
